## Mother Machine Cluster Code

In [ ]:
import trenchripper as tr

#### Optionally transfer files into the scratch folder

In [ ]:
sourcedir = "/n/files/SysBio/PAULSSON\ LAB/Daniel/Image_Data/Bacillus_Project/2019-06-18_DE85_training_data"
targetdir = "/n/scratch2/de64/2019-06-18_DE85_training_data"
tr.cluster.transferjob(sourcedir, targetdir)

#### Start Dask

In [ ]:
# kymo_controller = tr.cluster.dask_controller(walltime='04:00:00',local=False,n_workers=40,memory='2GB')
dask_controller = tr.cluster.dask_controller(
    walltime="04:00:00", local=False, n_workers=40, memory="4GB"
)
dask_controller.startdask()
dask_controller.daskcluster.start_workers()

In [ ]:
dask_controller.daskclient

#### Perform Extraction

In [ ]:
# hdf5_extractor = tr.ndextract.hdf5_fov_extractor("/n/scratch2/de64/full_pipeline_test/40x_Ph2_Test_1.5.nd2","/n/scratch2/de64/full_pipeline_test")
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    "/n/scratch2/de64/2019-05-31_Bacillus_training_data_mneongreen/Main_Experiment.nd2",
    "/n/scratch2/de64/2019-05-31_Bacillus_training_data_mneongreen",
)

In [ ]:
hdf5_extractor.fov_metadata[:5]

In [ ]:
print(hdf5_extractor.exp_metadata["fields_of_view"])

In [ ]:
fov_list = list(range(0, 40))

In [ ]:
hdf5_extractor.writemetadata()

In [ ]:
dask_controller.mapfovs(hdf5_extractor.extract_fov, fov_list)

In [ ]:
hdf5_extractor.extract_fov(4)

#### Generate Kymographs

To begin, you should test which parameters work best on your dataset in the following [notebook](./Interactive_Kymograph.ipynb).

#### Start Dask

In [ ]:
dask_controller = tr.cluster.dask_controller(
    walltime="04:00:00", local=False, n_workers=40, memory="4GB"
)
dask_controller.startdask()
dask_controller.daskcluster.start_workers()

In [ ]:
dask_controller.daskclient

In [ ]:
# headpath = "/n/scratch2/de64/full_pipeline_test"
# all_channels = ["channel_RFP","channel_BF"]

# kymo = tr.kymograph.kychunker(headpath=headpath,all_channels=all_channels,trench_len_y=270,padding_y=20,trench_width_x=25,\
#                  t_chunk=100,t_range=(0,-1),y_percentile=88,y_min_edge_dist=50,smoothing_kernel_y=(17,1),triangle_nbins=50,triangle_scaling=1.6,\
#                  expected_num_rows=2,orientation_detection=0,orientation_on_fail=0,x_percentile=85,background_kernel_x=(21,1),smoothing_kernel_x=(9,1),otsu_nbins=50,otsu_scaling=0.3,\
#                  trench_present_thr=0.7)

headpath = "/n/scratch2/de64/2019-05-31_Bacillus_training_data_mneongreen/"
all_channels = ["channel_GFP", "channel_Phase"]

kymo = tr.kymograph.kychunker(
    headpath=headpath,
    all_channels=all_channels,
    trench_len_y=250,
    padding_y=40,
    trench_width_x=23,
    t_chunk=100,
    t_range=(0, None),
    y_percentile=88,
    y_min_edge_dist=80,
    smoothing_kernel_y=(17, 1),
    triangle_nbins=50,
    triangle_scaling=1.05,
    expected_num_rows=2,
    orientation_detection=0,
    orientation_on_fail=0,
    x_percentile=80,
    background_kernel_x=(21, 1),
    smoothing_kernel_x=(9, 1),
    otsu_nbins=50,
    otsu_scaling=0.5,
    trench_present_thr=0.7,
)

In [ ]:
# Y Percentile 88
# Y Smoothing Kernel 17
# Triangle Trheshold Bins 50
# Triangle Threshold Scaling 1.05
# Minimum Trench Length 50
# Y Padding 20
# Trench Length 270
# Orientation Detection Method 0
# Expected Number of Rows (Manual Orientation Detection) 2
# Top Orientation when Row Drifts Out (Manual Orientation Detection) 0
# X Percentile 85
# X Background Kernel 21
# X Smoothing Kernel 9
# Otsu Trheshold Bins 50
# Otsu Threshold Scaling 0.3
# Trench Width 25
# Trench Presence Threshold 0.7

In [ ]:
kymo.dask_full_kymograph(dask_controller)

In [ ]:
kymo.kymo_report()

In [ ]:
# fov_list = list(range(0,80))
# fov_list = list(range(0,40))
# dask_controller.mapfovs(kymo.generate_kymograph,fov_list,retries=1)
# kymo.generate_kymograph(30)
# kymo.collect_metadata(fov_list,use_archive=False,overwrite_archive=True)
# kymo.collect_metadata(fov_list,use_archive=True,overwrite_archive=False)
# dask_controller.mapfovs(kymo.reorg_kymographs,fov_list,retries=1)
# kymo.cleanup_kymographs()

maybe an estimate of memory usage

#### Generate Segmentation

To begin, you should test which parameters work best on your dataset in the following [notebook](./Interactive_fluo_seg.ipynb).

#### Start Dask

In [ ]:
dask_controller = tr.cluster.dask_controller(
    walltime="01:00:00", local=False, n_workers=40, memory="1GB"
)
dask_controller.startdask()
dask_controller.daskcluster.start_workers()

In [ ]:
dask_controller.daskclient

In [ ]:
# segment = tr.segment.fluo_segmentation_cluster("/n/scratch2/de64/full_pipeline_test","channel_RFP",smooth_sigma=0.75,wrap_pad=3,hess_pad=4,min_obj_size=30,cell_mask_method='local',\
#                  cell_otsu_scaling=1.,local_otsu_r=15,edge_threshold_scaling=1.,threshold_step_perc=0.05,threshold_perc_num_steps=4,convex_threshold=0.8)
segment = tr.segment.fluo_segmentation_cluster(
    "/n/scratch2/de64/2019-05-31_Bacillus_training_data_mneongreen/",
    "channel_GFP",
    smooth_sigma=0.75,
    wrap_pad=3,
    hess_pad=4,
    min_obj_size=50,
    cell_mask_method="local",
    global_otsu_scaling=0.9,
    cell_otsu_scaling=0.7,
    local_otsu_r=11,
    edge_threshold_scaling=0.9,
    threshold_step_perc=0.05,
    threshold_perc_num_steps=3,
    convex_threshold=0.75,
)

In [ ]:
# Scale Fluorescence? True
# Scaling Percentile: 90
# Gaussian Kernel Sigma: 0.75
# Cell Mask Thresholding Method: local
# Global Threshold Scaling: 0.9
# Cell Threshold Scaling: 0.7
# Local Otsu Radius: 11
# Edge Threshold Scaling: 0.9
# Threshold Step Percent: 0.05
# Number of Threshold Steps: 3
# Minimum Object Size: 30
# Convexity Threshold: 0.75

In [ ]:
# fov_list = list(set(range(0,80))-set([77]))
fov_list = list(set(range(0, 40)) - set([32, 14, 25, 26, 29]))

In [ ]:
dask_controller.mapfovs(segment.generate_segmentation, fov_list)

In [ ]:
segment.generate_segmentation(1)

In [ ]:
kymo_controller.daskclient.get_futures_error(kymo_controller.futures[1])

In [ ]:
kymo_handle.return_unwrap()

In [ ]:
import h5py
from matplotlib import pyplot as plt
import numpy as np
import trenchripper as tr

kymo_handle = tr.utils.kymo_handle()

with h5py.File(
    "/n/scratch2/de64/2019-05-31_Bacillus_training_data_mneongreen/kymo/kymo_12.hdf5",
    "r",
) as infile:
    print(infile.keys())
    print(infile["1020"])
    kymo_handle.import_wrap(infile["1020" + "/channel_GFP"])
    plt.imshow(infile["1020" + "/channel_GFP"][:, :, 0])
    plt.show()
    plt.imshow(kymo_handle.return_unwrap())
    plt.show()

In [ ]:
import h5py
from matplotlib import pyplot as plt
import numpy as np

kymo_handle = tr.utils.kymo_handle()

with h5py.File(
    "/n/scratch2/de64/2019-05-31_Bacillus_training_data_mneongreen/segmentation/seg_12.hdf5",
    "r",
) as infile:
    print(infile.keys())
    kymo_handle.import_wrap(infile["1020"])
    plt.imshow(infile["1020"][:, :, 0])
    plt.show()
    plt.imshow(kymo_handle.return_unwrap())
    plt.show()

### Note
There may be an issue where hdf5 files do not close properly on crashes...

In [ ]:
# props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

#### Dask Utilities

In [ ]:
dask_controller.shutdown()

In [ ]:
dask_controller.retry_failed()

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
dask_controller.retry_processing()

In [ ]:
# scancel --name=dask-worker --user=de64

#### Note: need to find a way to associate lanes from init barcoding snapshot with imaging lanes

plan:
- extract and segment (with phase) barcoding file
- extract and segment experiemnt file
- for both, note x positions of t=0 trenches
- compare these values to map trenches to eachother...